In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
df = pd.read_csv("data/usa/presidential_speeches.csv")

In [4]:
df[df["Date"]>"2019-01-01"]

,Date,President,Party,Speech Title,Summary,Transcript,URL
987,2019-01-19,Donald Trump,Republican,Remarks about the US Southern Border,President Donald Trump speaks about what he se...,"Just a short time ago, I had the honor of pres...",https://millercenter.org/the-presidency/presid...
988,2019-02-05,Donald Trump,Republican,State of the Union Address,"In his second State of the Union Address, Pres...","Madam Speaker, Mr. Vice President, Members of ...",https://millercenter.org/the-presidency/presid...
989,2019-02-15,Donald Trump,Republican,Speech Declaring a National Emergency,President Donald Trump declares a national eme...,"Thank you very much, everybody. Before we begi...",https://millercenter.org/the-presidency/presid...
990,2019-09-24,Donald Trump,Republican,Remarks at the United Nations General Assembly,President Donald Trump speaks to the 74th sess...,"Thank you very much. Mr. President, Mr. Secret...",https://millercenter.org/the-presidency/presid...
991,2019-09-25,Donald Trump,Republican,Press Conference,President Donald Trump holds a press conferenc...,"Thank you very much. Thank you. Well, thank yo...",https://millercenter.org/the-presidency/presid...


In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests

In [6]:
driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')
soup = BeautifulSoup(driver.page_source, 'html.parser')

/var/folders/93/9kgm8vks4wz6zr8rg0d_pb7w0000gn/T/ipykernel_26786/3002165147.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = '/usr/local/bin/geckodriver')


In [7]:
driver.get('https://millercenter.org/the-presidency/presidential-speeches')

In [8]:
for i in tqdm(range(150)):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    
    #Wait for page to load
    time.sleep(1)

100%|██████████| 150/150 [02:32<00:00,  1.01s/it]


In [9]:
speeches = []

soup1 = BeautifulSoup(driver.page_source, 'html.parser')
rows1 = soup1.find_all('div', class_ = 'views-row')
for x in rows1:
    endings = x.find('a')['href'][38:]
    speeches.append(endings)

In [24]:
base_url = 'https://millercenter.org/the-presidency/presidential-speeches/'
presidents = {}
for i in tqdm(speeches):
    full_url = base_url + i
    response_all = requests.get(full_url)
    soup_all = BeautifulSoup(response_all.text,'html.parser')
    name = soup_all.find(class_ = 'president-name').text
    date = soup_all.find(class_ = 'episode-date').text
    if date == "September 25, 2019":
        break
    transcript = soup_all.find(class_="view-transcript").text  
    
    presidents[i] = [name] + [date] + [transcript]


  4%|▍         | 41/1033 [02:08<52:01,  3.15s/it] 


In [25]:
df_dict = pd.DataFrame(presidents)
df_uva = df_dict.T
df_uva.columns = ['President', 'Date', 'Speech']
df_uva.tail(3)

,President,Date,Speech
january-8-2020-statement-iran,Donald Trump,"January 08, 2020",\n\n\nView Transcript\n \n\n\n\n\nTranscr...
january-3-2020-remarks-killing-qasem-soleimani,Donald Trump,"January 03, 2020",\n\n\nView Transcript\n \n\n\n\n\nTranscr...
october-27-2019-statement-death-abu-bakr-al-baghdadi,Donald Trump,"October 27, 2019",\n\n\nView Transcript\n \n\n\n\n\nTranscr...


In [27]:
def clean(text):
    text = text.replace("\n\n\nView Transcript\n      \n\n\n\n\nTranscript\n", "")
    text = text.replace("\r", "").replace("\n", " ").replace("\t", " ").replace("\xa0", "")
    text = text.replace("")
    return text

In [28]:
df_uva['Speech'] = df_uva['Speech'].apply(clean)

In [30]:
df = pd.read_csv("data/usa/presidential_speeches.csv")

In [33]:
df = df[["President", "Date", "Transcript"]]

In [34]:
df

,President,Date,Transcript
0,George Washington,1789-04-30,Fellow Citizens of the Senate and the House of...
1,George Washington,1789-10-03,Whereas it is the duty of all Nations to ackno...
2,George Washington,1790-01-08,Fellow Citizens of the Senate and House of Rep...
3,George Washington,1790-12-08,Fellow citizens of the Senate and House of Rep...
4,George Washington,1790-12-29,"I the President of the United States, by my ow..."
...,...,...,...
987,Donald Trump,2019-01-19,"Just a short time ago, I had the honor of pres..."
988,Donald Trump,2019-02-05,"Madam Speaker, Mr. Vice President, Members of ..."
989,Donald Trump,2019-02-15,"Thank you very much, everybody. Before we begi..."
990,Donald Trump,2019-09-24,"Thank you very much. Mr. President, Mr. Secret..."


In [39]:
# reverse df
df = df.iloc[::-1]

In [35]:
df_uva = df_uva[["President", "Date", "Speech"]]
df_uva.rename(columns = {'Speech':'Transcript'}, inplace = True)

In [37]:
df_uva.reset_index(drop=True, inplace=True)

In [41]:
df_uva.head()

,President,Date,Transcript
0,Joe Biden,"September 01, 2022","THE PRESIDENT: My fellow Americans, please, if..."
1,Joe Biden,"May 24, 2022","Good evening, fellow Americans. I had hoped, w..."
2,Joe Biden,"March 26, 2022","THE PRESIDENT: Thank you, thank you, thank you..."
3,Joe Biden,"March 01, 2022",SPEAKER OF THE HOUSE NANCY PELOSI: Members of ...
4,Joe Biden,"February 24, 2022",THE PRESIDENT: Sorry to keep you waiting. Good...


In [44]:
def get_month(month):
    month = month.lower()
    if month == "january" or month == "jan":
        return "01"
    elif month == "february" or month == "feb":
        return "02"
    elif month == "march" or month == "mar":
        return "03"
    elif month == "april" or month == "apr":
        return "04"
    elif month == "may":
        return "05"
    elif month == "june" or month == "jun":
        return "06"
    elif month == "july" or month == "jul":
        return "07"
    elif month == "august" or month == "aug":
        return "08"
    elif month == "september" or month == "sep":
        return "09"
    elif month == "october" or month == "oct":
        return "10"
    elif month == "november" or month == "nov":
        return "11"
    elif month == "december" or month == "dec":
        return "12"
    else:
        return "01"


def parse_date(date):
    month = date.split(" ")[0]
    day = date.split(" ")[1].split(",")[0]
    year = date.split(" ")[2]

    month = get_month(month)
    day = day.zfill(2)

    return f"{day}/{month}/{year}"

In [45]:
df_uva["Date"] = df_uva["Date"].apply(parse_date)

In [50]:
df_uva["Date"] = pd.to_datetime(df_uva["Date"], format="%d/%m/%Y")

In [51]:
final_df = pd.concat([df_uva, df], axis=0)
final_df.reset_index(drop=True, inplace=True)

In [52]:
final_df

,President,Date,Transcript
0,Joe Biden,2022-09-01 00:00:00,"THE PRESIDENT: My fellow Americans, please, if..."
1,Joe Biden,2022-05-24 00:00:00,"Good evening, fellow Americans. I had hoped, w..."
2,Joe Biden,2022-03-26 00:00:00,"THE PRESIDENT: Thank you, thank you, thank you..."
3,Joe Biden,2022-03-01 00:00:00,SPEAKER OF THE HOUSE NANCY PELOSI: Members of ...
4,Joe Biden,2022-02-24 00:00:00,THE PRESIDENT: Sorry to keep you waiting. Good...
...,...,...,...
1028,George Washington,1790-12-29,"I the President of the United States, by my ow..."
1029,George Washington,1790-12-08,Fellow citizens of the Senate and House of Rep...
1030,George Washington,1790-01-08,Fellow Citizens of the Senate and House of Rep...
1031,George Washington,1789-10-03,Whereas it is the duty of all Nations to ackno...


In [53]:
final_df.to_csv("data/usa/final_presidential_speeches.csv", index=False)